In [1]:
import torch
import torch.nn as nn

import numpy as np

from functorch import make_functional, vmap, grad, jacrev
import functools

from pyDOE import lhs

device = torch.device('cpu')

torch.set_default_dtype(torch.float64)

In [2]:
class Plain(nn.Module):
    
    def __init__(self, in_dim , h_dim , out_dim):
        super().__init__()
        self.ln1 = nn.Linear( in_dim , h_dim )
        self.act1 =nn.Sigmoid()
        self.ln2 = nn.Linear( h_dim , out_dim)
        
    def forward(self, x):
        out = self.ln1(x)
        out = self.act1(out)
        out = self.ln2(out)
        return out

In [3]:
def compute_loss_Res(func_params, X_inner, Rf_inner):

    def f(x, func_params):
        output = func_model(func_params, x)
        return output.squeeze(0)
    
    grad2_f = (jacrev(grad(f)))(X_inner, func_params)
    dudX2 = (torch.diagonal(grad2_f))
    
    laplace = (dudX2[0] + dudX2[1] + dudX2[2])
    
    loss_Res = laplace - Rf_inner

    return loss_Res.flatten()


def compute_loss_b(func_params, X_bd, U_bd):

    def f(x, func_params):
        output = func_model(func_params, x)
        return output.squeeze(0)
    
    u_pred = f(X_bd, func_params)
    loss_b = u_pred - U_bd
        
    return loss_b.flatten()

def compute_loss_j(func_params, X_ij, Uj_ij):

    def f(x, func_params):
        output = func_model(func_params, x)
        return output.squeeze(0)
    
    X_ij=X_ij.reshape(len(X_ij), 1)

    ij_outer = torch.cat((X_ij[0], X_ij[1], X_ij[2], 1.0+0.0*X_ij[0]), 0)
    ij_inner = torch.cat((X_ij[0], X_ij[1], X_ij[2], -1.0+0.0*X_ij[0]), 0)

    u_ij_outer = f(ij_outer, func_params)
    u_ij_inner = f(ij_inner, func_params)
    
    ij_pred = u_ij_outer - u_ij_inner
    
    loss_j = ij_pred - Uj_ij
        
    return loss_j.flatten()

def compute_loss_normal_jump(func_params, X_ij, Unj_ij):

    def f(x, func_params):
        output = func_model(func_params, x)
        return output.squeeze(0)
    
    X_ij=X_ij.reshape(len(X_ij), 1)
    
    ij_outer = torch.cat((X_ij[0], X_ij[1], X_ij[2], 1.0+0.0*X_ij[0]), 0)
    ij_inner = torch.cat((X_ij[0], X_ij[1], X_ij[2], -1.0+0.0*X_ij[0]), 0)

    grad_f_outer = (grad(f))(ij_outer, func_params)
    df_outer = (grad_f_outer)
    Normal_outer = (df_outer[0]*X_ij[0]/49.0 + df_outer[1]*X_ij[1]/25.0 + df_outer[2]*X_ij[2]/9.0)/torch.sqrt((X_ij[0]/49.0)**2 + (X_ij[1]/25.0)**2 + (X_ij[2]/9.0)**2)
    grad_f_inner = (grad(f))(ij_inner, func_params)
    df_inner = (grad_f_inner)
    Normal_inner = (df_inner[0]*X_ij[0]/49.0 + df_inner[1]*X_ij[1]/25.0 + df_inner[2]*X_ij[2]/9.0)/torch.sqrt((X_ij[0]/49.0)**2 + (X_ij[1]/25.0)**2 + (X_ij[2]/9.0)**2)
    
    normal_jump_pred = 1.0e-3*Normal_outer - Normal_inner

    loss_normal_jump = normal_jump_pred - Unj_ij
        
    return loss_normal_jump.flatten()

In [4]:
def exact_u(X):
  z = X[:, 3:4]
  # exact_u = exact solution
  # if z= 1 -> u1
  u1 = np.sin(X[:,0:1])*np.sin(X[:,1:2])*np.sin(X[:,2:3])
  # if z=-1 -> fn1
  un1 = np.exp(X[:,0:1]+X[:,1:2]+X[:,2:3])
  eu = u1*(z+1.0)/2.0 + un1*(1.0-z)/2.0
  return eu

def rhs_f(x):
  # if z= 1 -> f1
  f1 = -3.0*np.sin(x[:,0:1])*np.sin(x[:,1:2])*np.sin(x[:,2:3])
  # if z=-1 -> fn1
  fn1 = 3.0*np.exp(x[:,0:1]+x[:,1:2]+x[:,2:3])
  f = f1*(x[:,3:4]+1.0)/2.0 + fn1*(1.0-x[:,3:4])/2.0
  return f

def normal_u(x0, x1, x2, z):
  # exact_u = exact solution
  # if z= 1 -> u1
  u1 = np.cos(x0)*np.sin(x1)*np.sin(x2)*x0/49.0\
  +np.sin(x0)*np.cos(x1)*np.sin(x2)*x1/25.0\
  +np.sin(x0)*np.sin(x1)*np.cos(x2)*x2/9.0
  dist = np.sqrt((x0/49.0)**2 + (x1/25.0)**2 + (x2/9.0)**2)
  u1 = u1/dist
  # if z=-1 -> fn1
  un1 = (x0/49.0+x1/25.0+x2/9.0)*np.exp(x0+x1+x2)
  un1 = un1/dist
  nu = u1*(z+1.0)/2.0 + un1*(1.0-z)/2.0
  return nu

def sign_x(X):
  z = 0.0*X[:,0:1] + 1.0
  for i in range(len(z)):
    dist = (X[i,0]/0.7)**2+(X[i,1]/0.5)**2+(X[i,2]/0.3)**2
    if dist < 1.0:
      z[i] = -1.0
  return z

In [5]:
def chebyshev_first_kind(dim,n):
  a_new=(1.0/n)-1.0
  X=[]
  x=[]
  X=(np.mgrid[[slice(None,n),]*dim])
  XX=np.cos(np.pi*(X+0.5)/n)
  for i in range(len(X)):
    x.append(np.array(XX[i].tolist()).reshape(n**dim,1))
  return np.hstack(np.array(x))

In [6]:
d=3

# number of grid points
N_inner = 216
N_bd = 36
N_ij = 3*N_bd

# Training points

## X_inner: points inside the domain, totally (N_inner-1)**2 points
X_inner = chebyshev_first_kind(d, 6)
z_inner = sign_x(X_inner)
X_inner = np.hstack([X_inner, z_inner])
Rf_inner = rhs_f(X_inner)
Rf_inner = Rf_inner.reshape(N_inner, 1)

## X_bd: points at the boundary, totally 2*d*N_bd points
xx = lhs(d-1, N_bd)*2.0-1.0
xx = np.hstack((-1.0*np.ones((N_bd,1)), xx))
yy = lhs(d-1, N_bd)*2.0-1.0
yy = np.hstack((np.ones((N_bd,1)), yy))
X_bd = np.vstack([xx, yy])

for i in range(1, d-1):
  xx1 = lhs(i, N_bd)*2.0-1.0
  xx2 = lhs(d-1-i, N_bd)*2.0-1.0
  xx = np.hstack((xx1, -1.0*np.ones((N_bd,1)), xx2))
  yy1 = lhs(i, N_bd)*2.0-1.0
  yy2 = lhs(d-1-i, N_bd)*2.0-1.0
  yy = np.hstack((yy1, np.ones((N_bd,1)), yy2))
  X_bd = np.vstack([X_bd, xx, yy])

xx = lhs(d-1, N_bd)*2.0-1.0
xx = np.hstack((xx, -1.0*np.ones((N_bd,1))))
yy = lhs(d-1, N_bd)*2.0-1.0
yy = np.hstack((yy, np.ones((N_bd,1))))
X_bd = np.vstack([X_bd, xx, yy])
X_bd = np.hstack((X_bd, 1.0+0.0*X_bd[:,0:1]))

## U_bd: function values at the boundary, totally 2*d*N_bd points
U_bd = exact_u(np.hstack([X_bd, 0.0*X_bd[:,0:1]+1.0]))
U_bd = U_bd.reshape(2*d*N_bd, 1)

## X_ij: points at the interior interface, totally N_bd points
r_ij = lhs(1,N_ij)*2.0*np.pi
p_ij = lhs(1,N_ij)*np.pi
x0_ij = 0.7*np.cos(r_ij)*np.sin(p_ij)
x1_ij = 0.5*np.sin(r_ij)*np.sin(p_ij)
x2_ij = 0.3*np.cos(p_ij)
X_ij  = np.hstack([x0_ij, x1_ij, x2_ij])

## Uj_ij: function jump at the interior interface, totally N_bd points
Uj_ij = exact_u(np.hstack([X_ij, 0.0*X_ij[:,0:1]+1.0])) - exact_u(np.hstack([X_ij, 0.0*X_ij[:,0:1]-1.0]))
Uj_ij = Uj_ij.reshape(N_ij, 1)

# beta_plus
beta_plus = 1.0e-3
## Unj_ij: normal jump at the interior interface, totally N_bd points
Unj_ij = beta_plus*normal_u(x0_ij, x1_ij, x2_ij, 0.0*x0_ij+1.0) - normal_u(x0_ij, x1_ij, x2_ij, 0.0*x0_ij-1.0)

In [7]:
# single-layer model
model = Plain(4, 30, 1).to(device)
print(model)

# Make model a functional
func_model, func_params = make_functional(model)

Plain(
  (ln1): Linear(in_features=4, out_features=30, bias=True)
  (act1): Sigmoid()
  (ln2): Linear(in_features=30, out_features=1, bias=True)
)


In [8]:
X_bd = torch.from_numpy(X_bd).requires_grad_(True).double().to(device)
U_bd = torch.from_numpy(U_bd).double().to(device)
X_inner = torch.from_numpy(X_inner).requires_grad_(True).double().to(device)
Rf_inner = torch.from_numpy(Rf_inner).double().to(device)
X_ij = torch.from_numpy(X_ij).requires_grad_(True).double().to(device)
Uj_ij = torch.from_numpy(Uj_ij).double().to(device)
Unj_ij = torch.from_numpy(Unj_ij).double().to(device)

In [9]:
LM_iter = 2000
mu_update = 2 # update \mu every mu_update iterations
div_factor = 1.3 # \mu <- \mu/div_factor when loss decreases
mul_factor = 3 # \mu <- mul_factor*\mu when loss incerases

mu = 10**5
loss_sum_old = 10**5
itera = 0

In [10]:
%%time
for step in range(LM_iter+1):
    # Put into loss functional to get L_vec
    L_vec_res = vmap(compute_loss_Res, (None, 0, 0))(func_params, X_inner, Rf_inner)
    L_vec_b = vmap(compute_loss_b, (None, 0, 0))(func_params, X_bd, U_bd)
    L_vec_j = vmap(compute_loss_j, (None, 0, 0))(func_params, X_ij, Uj_ij)
    L_vec_nj = vmap(compute_loss_normal_jump, (None, 0, 0))(func_params, X_ij, Unj_ij)

    L_vec_res = L_vec_res/np.sqrt(N_inner)
    L_vec_b = L_vec_b/np.sqrt(N_inner)
    L_vec_j = L_vec_j/np.sqrt(3.0*N_bd)
    L_vec_nj = L_vec_nj/np.sqrt(3.0*N_bd)
    loss = torch.sum(L_vec_res**2) + torch.sum(L_vec_b**2) + torch.sum(L_vec_j**2) + torch.sum(L_vec_nj**2)

    # Consturct J for domain points
    # (None, 0 ,0): func_params: no batch. data_d: batch wrt shape[0] (data[i, :]). force_value: batch wrt shape[0] (force_value[i,:])
    
    per_sample_grads = vmap(jacrev(compute_loss_Res), (None, 0, 0))(func_params, X_inner, Rf_inner)
    cnt = 0
    for g in per_sample_grads: 
        g = g.detach()
        J_d_res = g.view(len(g), -1) if cnt == 0 else torch.hstack([J_d_res, g.view(len(g), -1)])
        cnt = 1
    
    per_sample_grads = vmap(jacrev(compute_loss_b), (None, 0, 0))(func_params, X_bd, U_bd)
    cnt = 0
    for g in per_sample_grads: 
        g = g.detach()
        J_d_b = g.view(len(g), -1) if cnt == 0 else torch.hstack([J_d_b, g.view(len(g), -1)])
        cnt = 1
        
    per_sample_grads = vmap(jacrev(compute_loss_j), (None, 0, 0))(func_params, X_ij, Uj_ij)
    cnt = 0
    for g in per_sample_grads: 
        g = g.detach()
        J_d_j = g.view(len(g), -1) if cnt == 0 else torch.hstack([J_d_j, g.view(len(g), -1)])
        cnt = 1
        
    per_sample_grads = vmap(jacrev(compute_loss_normal_jump), (None, 0, 0))(func_params, X_ij, Unj_ij)
    cnt = 0
    for g in per_sample_grads: 
        g = g.detach()
        J_d_nj = g.contiguous().view(len(g), -1) if cnt == 0 else torch.hstack([J_d_nj, g.view(len(g), -1)])
        cnt = 1

    # cat J_d and J_b into J
    J_mat = torch.cat((J_d_res, J_d_b, J_d_j, J_d_nj))
    L_vec = torch.cat((L_vec_res, L_vec_b, L_vec_j, L_vec_nj))

    # update lambda
    I = torch.eye((J_mat.shape[1])).to(device)

    with torch.no_grad():
        J_product = J_mat.t()@J_mat
        rhs = -J_mat.t()@L_vec
        with torch.no_grad():
            dp = torch.linalg.solve(J_product + mu*I, rhs)

        # update parameters
        cnt=0
        for p in func_params:
            mm=torch.Tensor([p.shape]).tolist()[0]
            num=int(functools.reduce(lambda x,y:x*y,mm,1))
            p+=dp[cnt:cnt+num].reshape(p.shape)
            cnt+=num

        itera += 1
        if step % mu_update == 0:
            #if loss_sum_check < loss_sum_old:
            if loss < loss_sum_old:
                mu = max(mu/div_factor, 10**(-9))
            else:
                mu = min(mul_factor*mu, 10**(8))
            loss_sum_old = loss
                
        if step%100 == 0:
            print(
                    'Iter %d, Loss_Res: %.5e, mu: %.5e' % (itera, loss.item(), mu)
                )

        if step == LM_iter or loss.item()<10**(-12):
            break

Iter 1, Loss_Res: 4.32054e+00, mu: 7.69231e+04
Iter 101, Loss_Res: 3.67593e-01, mu: 1.54486e-01
Iter 201, Loss_Res: 1.41829e-05, mu: 4.71901e-06
Iter 301, Loss_Res: 1.71287e-07, mu: 1.97817e-06
Iter 401, Loss_Res: 5.00307e-08, mu: 1.26127e-05
Iter 501, Loss_Res: 2.03807e-08, mu: 5.28715e-06
Iter 601, Loss_Res: 1.19812e-08, mu: 8.64370e-06
Iter 701, Loss_Res: 7.60825e-09, mu: 1.41312e-05
Iter 801, Loss_Res: 3.18604e-09, mu: 5.92369e-06
Iter 901, Loss_Res: 1.29649e-09, mu: 2.48317e-06
Iter 1001, Loss_Res: 9.94227e-10, mu: 1.04093e-06
Iter 1101, Loss_Res: 8.45260e-10, mu: 4.36349e-07
Iter 1201, Loss_Res: 7.53052e-10, mu: 7.13366e-07
Iter 1301, Loss_Res: 6.71532e-10, mu: 2.99038e-07
Iter 1401, Loss_Res: 6.16106e-10, mu: 4.88882e-07
Iter 1501, Loss_Res: 5.71979e-10, mu: 2.04936e-07
Iter 1601, Loss_Res: 5.26540e-10, mu: 8.59077e-08
Iter 1701, Loss_Res: 4.78344e-10, mu: 1.40446e-07
Iter 1801, Loss_Res: 4.35230e-10, mu: 2.29609e-07
Iter 1901, Loss_Res: 4.12652e-10, mu: 3.75377e-07
Iter 2001, L

In [11]:
N_test = 2000

X_inn = lhs(d, N_test)*2.0-1.0
z_inn = sign_x(X_inn)
X_inn = np.hstack([X_inn, z_inn])
Exact_test = exact_u(X_inn)
Exact_test = Exact_test.reshape((N_test,1))
X_inn_torch = torch.tensor(X_inn).double().to(device)
u_pred = func_model(func_params, X_inn_torch).detach().cpu().numpy()

error = np.absolute(u_pred - Exact_test)

error_u_inf = np.linalg.norm(error, np.inf)
print('Error u (absolute inf-norm): %e' % (error_u_inf))
error_u_2 = np.linalg.norm(error,2)/np.sqrt(N_test)
print('Error u (absolute 2-norm): %e' % (error_u_2))

Error u (absolute inf-norm): 3.215130e-05
Error u (absolute 2-norm): 2.480416e-06
